# Fine-Tune FunctionGemma using Hugging Face TRL and OpenEnv

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/huggingface/trl/blob/main/examples/notebooks/grpo_functiongemma_browsergym_openenv.ipynb)

![trl banner](https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/trl_banner_dark.png)

This guide describes the process of fine-tuning [FunctionGemma](https://huggingface.co/google/functiongemma-270m-it) by Google DeepMind in the [BrowserGym](https://meta-pytorch.org/OpenEnv/environments/browsergym/) environment provided by OpenEnv, using Hugging Face TRL. The steps covered include:

* What is GRPO and OpenEnv
* Setup dependencies for training
* Initialize the OpenEnv's BrowserGym environment
* Create rollout function with helpers
* Define the reward functions
* Load the custom dataset
* Fine tune using TRL and the GRPOTrainer
* Load the fine-tuned model and run inference

> Note: The guide is designed to run on Google Colaboratory with access to an NVIDIA A100 GPU (40GB) using FunctionGemma. The workflow can be adapted to other GPU configurations, models, or environments.

## What is GRPO and OpenEnv

Group Relative Policy Optimization ([GRPO](https://huggingface.co/papers/2402.03300)) is a post-training method widely used for efficiently fine-tuning large language models. GRPO leverages reward functions to guide learning, enabling models to optimize task-specific behaviors without retraining the entire network.

[OpenEnv](https://meta-pytorch.org/OpenEnv) provides a standard interface for interacting with agentic execution environments using simple Gymnasium-style APIs, such as `step()`, `reset()`, and `state()`. These APIs facilitate reinforcement learning training loops by allowing models to interact with environments in a structured manner. OpenEnv also offers tools for environment creators to build isolated, secure, and deployable environments that can be shared via common protocols like HTTP or packaged in Docker.

The combination of GRPO and OpenEnv enables efficient fine-tuning of models in controlled, interactive tasks while minimizing resource requirements.

## Setup dependencies for training

Install the required libraries, including Hugging Face TRL for fine-tuning and OpenEnv for reinforcement learning environments.

In [ ]:
#!pip install -Uq trl[vllm] git+https://github.com/meta-pytorch/OpenEnv.git openenv_core liger-kernel trackio
# Hotfix: https://github.com/huggingface/trl/pull/4740
!pip install -Uq trl[vllm] git+https://github.com/meta-pytorch/OpenEnv.git@bf5e968286e0d49cdc03fd904d48faff4b15a437 openenv_core==0.1.1 liger-kernel trackio

A valid Hugging Face token is required to save the fine-tuned model. In Google Colab, the token can be securely accessed through Colab secrets. Otherwise, it can be provided directly in the login method. Ensure the token has write permissions to allow uploading the model to the Hugging Face Hub during training.

In [ ]:
from google.colab import userdata
from huggingface_hub import login

# Login into Hugging Face Hub
hf_token = userdata.get('HF_TOKEN') # If you are running inside a Google Colab
login(hf_token)

## Initialize the OpenEnv's BrowserGym environment

External environments can guide the fine-tuning of LLMs for function calling by providing interactive feedback that enhances performance on task-specific behaviors.

[BrowserGym](https://meta-pytorch.org/OpenEnv/environments/browsergym/) is a unified framework for web-based agent tasks, offering multiple benchmarks through a Gymnasium-compatible API. It enables training on simple synthetic tasks with [MiniWoB++](https://github.com/Farama-Foundation/miniwob-plusplus) and evaluation on more complex, realistic tasks with [WebArena](https://github.com/web-arena-x/webarena), [VisualWebArena](https://github.com/web-arena-x/visualwebarena), or [WorkArena](https://github.com/ServiceNow/WorkArena). This setup supports iterative training and assessment of web agents without requiring extensive infrastructure.

BrowserGym supports both LLM and VLM training by providing visual information, including screenshots and DOM data, which can be utilized depending on the model type. This guide focuses on a simple web-based task called *"click-test"*, which is part of the MiniWoB++ benchmark of synthetic web tasks. Environments can be run locally, in Docker containers, or accessed remotely via the Hugging Face Hub. For this example, the remote environment [burtenshaw/browsergym-v2](https://huggingface.co/spaces/burtenshaw/browsergym-v2) will be used.

> Note: Hosted environments on the Hub currently have limited concurrency. For higher reliability or parallel runs, duplicating the Space to your own account is strongly recommended.

In [ ]:
from envs.browsergym_env import BrowserGymEnv
space_url = "https://burtenshaw-browsergym-v2.hf.space"

client = BrowserGymEnv(base_url=space_url)

## Create rollout function with helpers

The rollout function defines how the agent interacts with the environment during GRPO training. It generates model outputs, collects feedback in the form of rewards, and returns the information required for optimization.

In this setup:
- The function is invoked automatically by the GRPOTrainer (introduced later), which orchestrates the training loop and handles policy updates.
- It uses the trainer's `generate_rollout_completions()` method for efficient output generation. This leverages vLLM, a high-performance inference engine for large language models, and is integrated within TRL to streamline rollout generation and reward collection during fine-tuning.
- Each rollout represents a complete interaction loop, where the model acts, receives feedback from the environment, and updates based on reward signals.

Rewards capture various aspects of the agent's performance. Helper functions, such as `rollout_once`, manage individual episodes, keeping the main `rollout_func` clean, modular, and reusable.

This modular structure allows GRPO to efficiently sample, evaluate, and refine the model's behavior through reinforcement learning.

Before executing rollouts, a `system prompt` is defined to instruct the model on how to interact with the environment. This prompt specifies the available BrowserGym actions (such as `click`, `fill`, `send_keys`, and `scroll`), describes the page structure, and enforces that the model responds with exactly one action per step. It ensures consistent and structured interactions, guiding the model to complete tasks effectively without providing extra explanations or multiple actions.

In [ ]:
# @title System prompt (click to expand)
SYSTEM_PROMPT = """You control a web browser through BrowserGym actions.
You must complete the given web task by interacting with the page.

Available actions:
- noop() - Do nothing
- click(bid) - Click element with BrowserGym ID (the number in brackets)
- fill(bid, text) - Fill input field with text
- send_keys(text) - Send keyboard input
- scroll(direction) - Scroll up/down

The page structure shows elements as: [bid] element_type 'element_text'
For example: [13] button 'Click Me!' means bid='13'

Reply with exactly ONE action on a single line, e.g.:
click('13')
fill('42', 'hello world')
noop()

Do not include explanations or multiple actions."""

The `rollout_func` orchestrates the interaction between the model and the remote BrowserGym environment. For each prompt in the batch, it executes a complete episode using the `rollout_once` function, collecting model outputs and rewards for GRPO optimization.

The parameter `max_steps` defines the maximum number of steps the model can take within a single episode. This limits the length of the interaction loop, ensuring that episodes terminate even if the task is not completed, and helps maintain efficient training.

During each episode, the function tracks prompt and completion IDs, log probabilities, and both step-wise and final rewards, returning them in a structured format for the trainer to perform policy updates.

In [ ]:
from trl import GRPOTrainer

max_steps=10

def rollout_func(prompts: list[str], trainer: GRPOTrainer) -> dict[str, list]:
    episode_prompt_ids: list[list[int]] = []
    episode_completion_ids: list[list[int]] = []
    episode_logprobs: list[list[float]] = []
    completion_rewards: list[float] = []

    print(f"\n[DEBUG] rollout_func called with {len(prompts)} prompts (LLM mode, text-only)")

    for i, prompt_text in enumerate(prompts):
        print(f"[DEBUG] Processing prompt {i + 1}/{len(prompts)}")
        episode = rollout_once(
            trainer=trainer,
            env=client,
            tokenizer=trainer.processing_class,
            dataset_prompt=prompt_text,
            max_steps=max_steps,
        )
        episode_prompt_ids.append(episode["prompt_ids"])
        episode_completion_ids.append(episode["completion_ids"])
        episode_logprobs.append(episode["logprobs"])
        completion_rewards.append(episode["completion_reward"])

    return {
        "prompt_ids": episode_prompt_ids,
        "completion_ids": episode_completion_ids,
        "logprobs": episode_logprobs,
        "completion_reward": completion_rewards,
    }

### Define `rollout_once`

The `rollout_once` function runs one complete interaction loop between the model and the BrowserGym environment using the trainer's generation method.  
It executes a single episode, from generating an action to receiving feedback and computing rewards.

Here's the step-by-step breakdown:

1. Environment reset: Start a new BrowserGym session and initialize the observation.
2. Prompt construction: Combine the system prompt, environment observation (text-only via the accessibility tree), and any relevant errors or state information to form the model input.
3. Generation: Use `trl.experimental.openenv.generate_rollout_completions()` to produce the model's action efficiently with vLLM.
4. Action parsing and execution: Interpret the model's output and execute the corresponding BrowserGym action (e.g., `click`, `fill`, `scroll`).
5. Reward calculation: Track step-wise rewards provided by the environment and compute completion rewards based on task success or failure.
6. Return structured rollout data: Includes prompt/completion IDs, log probabilities, step rewards, and the final reward for the episode.

This modular design allows each episode to be processed independently while providing rich feedback for the GRPO training loop, supporting both task completion and intermediate reward shaping.

In [ ]:
from trl.experimental.openenv import generate_rollout_completions
from envs.browsergym_env import BrowserGymAction
from transformers import AutoTokenizer

def rollout_once(
    trainer: GRPOTrainer,
    env: BrowserGymEnv,
    tokenizer: AutoTokenizer,
    dataset_prompt: str,
    max_steps: int,
) -> dict[str, list]:
    """Run one episode and collect training data (text-only, no screenshots)."""
    result = env.reset()
    observation = result.observation

    prompt_ids: list[int] = []
    completion_ids: list[int] = []
    logprobs: list[float] = []
    step_rewards: list[float] = []
    completion_rewards: list[float] = []

    for step_num in range(max_steps):
        if result.done:
            break

        # Create prompt from observation (text-only using accessibility tree)
        goal = observation.goal or dataset_prompt
        axtree = observation.axtree_txt or ""
        error = observation.error if observation.last_action_error else ""

        user_prompt = make_user_prompt(goal, step_num, axtree, error)
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ]
        prompt_text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False,
        )

        # Generate action with vLLM
        rollout_outputs = generate_rollout_completions(trainer, [prompt_text])[0]
        prompt_ids.extend(rollout_outputs["prompt_ids"])
        completion_ids.extend(rollout_outputs["completion_ids"])
        logprobs.extend(rollout_outputs["logprobs"])

        completion_text = rollout_outputs.get("text") or tokenizer.decode(
            rollout_outputs["completion_ids"], skip_special_tokens=True
        )

        # Parse and execute action
        action_str = parse_action(completion_text)

        print(f"Step {step_num + 1}: {action_str}")

        # Take action in environment
        result = env.step(BrowserGymAction(action_str=action_str))
        observation = result.observation

        # Track rewards
        step_reward = float(result.reward or 0.0)
        step_rewards.append(step_reward)

        # Reward shaping: success is most important
        if result.done and step_reward > 0:
            completion_rewards.append(1.0)  # Task completed successfully
        elif result.done and step_reward == 0:
            completion_rewards.append(0.0)  # Task failed
        else:
            completion_rewards.append(step_reward)  # Intermediate reward

    # Final reward is based on task completion
    final_reward = completion_rewards[-1] if completion_rewards else 0.0

    return {
        "prompt_ids": prompt_ids,
        "completion_ids": completion_ids,
        "logprobs": logprobs,
        "step_rewards": step_rewards,
        "completion_reward": final_reward,
    }

### Helper functions

Supporting utilities used in `rollout_once`:

- `make_user_prompt`: builds the user prompt combining the base text and previous game messages.
- `parse_action`: parses BrowserGym action from model response

In [ ]:
# @title Helpers (click to expand)
def make_user_prompt(goal: str, step_num: int, axtree: str, error: str = "") -> str:
    """Create user prompt from observation."""
    prompt_parts = [f"Step {step_num + 1}"]

    if goal:
        prompt_parts.append(f"Goal: {goal}")

    if error:
        prompt_parts.append(f"Previous action error: {error}")

    # Include accessibility tree (truncated for context)
    if axtree:
        max_len = 2000
        axtree_truncated = axtree[:max_len] + "..." if len(axtree) > max_len else axtree
        prompt_parts.append(f"Page structure:\n{axtree_truncated}")

    prompt_parts.append("What action do you take?")

    return "\n\n".join(prompt_parts)


def parse_action(response_text: str) -> str:
    """Parse BrowserGym action from model response."""
    # Extract first line that looks like an action
    for line in response_text.strip().split("\n"):
        line = line.strip()
        if "(" in line and ")" in line:
            return line

    # Fallback to noop if no valid action found
    return "noop()"

## Define the reward functions

Reward functions quantify the model's performance in the environment and guide the GRPO optimization process.

In this setup, the `reward_completion` function assigns rewards based on task completion. It extracts the final reward for each episode, which indicates whether the agent successfully completed the task. If no reward information is available, it defaults to zero.

This modular approach allows additional reward functions to be added easily, enabling more granular feedback such as intermediate progress, efficiency, or correctness of actions, depending on the task requirements.

In [ ]:
def reward_completion(completions: list[str], **kwargs) -> list[float]:
    """Reward for task completion."""
    rewards = kwargs.get("completion_reward") if kwargs else None
    if rewards is None:
        return [0.0 for _ in completions]
    return [float(r) for r in rewards]

## Load the custom dataset

The dataset is constructed with repeated prompts to control the total number of training episodes.

Each entry in the dataset triggers a single rollout episode during training. The `dataset_prompt` provides the initial instruction to the model at the start of each episode, ensuring consistent guidance for task execution.

In [ ]:
from datasets import Dataset

dataset_prompt = "Complete the web task successfully."
dataset_size = 1000

dataset = Dataset.from_dict({"prompt": [dataset_prompt] * dataset_size})

## Fine-tune using TRL and the GRPOTrainer

The next step is to define the GRPOConfig, which sets all key training parameters.

This configuration determines how the model interacts with vLLM, handles memory and computation, and records training metrics and logs for monitoring the fine-tuning process.

In [ ]:
from trl import GRPOConfig
output_dir = "browsergym-grpo-functiongemma-270m-it"

grpo_config = GRPOConfig(
    # num_train_epochs=1,                                     # Number of times to iterate over the full dataset (use for full training runs)
    max_steps=100,                                            # Number of dataset passes (for shorter runs/testing). For full trainings, use `num_train_epochs` instead
    learning_rate=5e-6,                                       # Learning rate for the optimizer
    warmup_steps=10,                                          # Number of steps to linearly increase learning rate at the start of training

    per_device_train_batch_size=1,                            # Number of samples per device per step
    num_generations=4,                                        # Number of completions to generate per prompt
    generation_batch_size=4,                                  # Batch size used during generation (must be divisible by num_generations)
    max_completion_length=32,                                 # Maximum length of generated completions

    use_vllm=True,                                            # Use vLLM engine for fast inference
    vllm_mode="colocate",                                     # vLLM mode: "colocate" runs generation on the same GPU as training
    vllm_gpu_memory_utilization=0.1,                          # Fraction of GPU memory allocated to vLLM

    output_dir=str(output_dir),                               # Directory where checkpoints, logs, and outputs will be saved
    logging_steps=1,                                          # Log metrics every N steps
    report_to="trackio",                                      # Logging/reporting platform (e.g., "trackio")
    trackio_space_id=output_dir,                              # HF Space where the experiment tracking will be saved
    push_to_hub=True,                                         # Optionally push trained model to Hugging Face Hub

    use_liger_kernel=True,                                    # Enable Liger kernel optimizations for faster training
    gradient_checkpointing=True,                              # Save memory by recomputing activations during backpropagation
    gradient_checkpointing_kwargs={"use_reentrant": False},   # Additional args to prevent warnings during gradient checkpointing
)


The next step is to initialize the GRPOTrainer, which manages the complete reinforcement learning loop.

It receives the model name, reward functions, rollout function, and dataset defined earlier. From the model name, the trainer automatically initializes the model and tokenizer. It then coordinates interactions between the model and the environment, applies the defined reward signals, and updates the policy during training.

Finally, calling `trainer.train()` starts the fine-tuning process, enabling the model to progressively improve its performance through iterative interaction and reinforcement learning.

> Note: The training pipeline uses approximately 10.6 GB of GPU VRAM and can be adapted to different hardware configurations.

In [ ]:
model_name = "google/functiongemma-270m-it"

In [ ]:
trainer = GRPOTrainer(
    model=model_name,
    reward_funcs=[reward_completion],
    train_dataset=dataset,
    args=grpo_config,
    rollout_func=rollout_func,
)

/tmp/ipython-input-3830121904.py:1: UserWarning: You are importing from 'rollout_func', which is an experimental feature. This API may change or be removed at any time without prior notice. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  trainer = GRPOTrainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 4/4 [00:00<00:00, 19.64it/s]


In [ ]:
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 2, 'pad_token_id': 0}.


* Trackio project initialized: huggingface
* Trackio metrics will be synced to Hugging Face Dataset: sergiopaniego/browsergym-grpo-functiongemma-270m-it-dataset
* Creating new space: https://huggingface.co/spaces/sergiopaniego/browsergym-grpo-functiongemma-270m-it
* View dashboard by going to: https://sergiopaniego-browsergym-grpo-functiongemma-270m-it.hf.space/


* Created new run: sergiopaniego-1765969078

[DEBUG] rollout_func called with 4 prompts (LLM mode, text-only)
[DEBUG] Processing prompt 1/4
Step 1: noop()
Step 2: noop()
Step 3: noop()
Step 4: noop()
Step 5: noop()
Step 6: noop()
Step 7: Click 'click(bid) - Click element with BrowserGym ID (the number in brackets
Step 8: I will use the action `click()` to click the button.
Step 9: noop()
Step 10: Click(bid) - Click element with BrowserGym ID (the number in brackets)
[DEBUG] Processing prompt 2/4
Step 1: noop()
Step 2: noop()
Step 3: Clicks ('13')
Step 4: I will click 'Click Me!' using action 'click(bid)' on page 'Click Test Task' using a bid of '13'.
Step 5: noop()
Step 6: noop()
Step 7: noop()
Step 8: noop()
Step 9: noop()
Step 10: noop()
[DEBUG] Processing prompt 3/4
Step 1: I will use the 'click(bid)' action.
Step 2: mouse_click(bid)
Step 3: click(bid) - Click element with BrowserGym ID (the number in brackets)
Step 4: Add action 'click(bid)' to Step 4.
Step 5: Click(bid) - Click el

/usr/local/lib/python3.12/dist-packages/torch/_inductor/compile_fx.py:282: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/_inductor/lowering.py:7095: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(


Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,-0.877900
10,1965.894400



[DEBUG] rollout_func called with 4 prompts (LLM mode, text-only)
[DEBUG] Processing prompt 1/4
Step 1: Clicks ('13')
Step 2: noop()
Step 3: noop()
Step 4: noop()
Step 5: noop()
Step 6: Click(bid) - Click element with BrowserGym ID (the number in brackets)
Step 7: noop()
Step 8: noop()
Step 9: click(bid) - Click element with BrowserGym ID (the number in brackets)
Step 10: noop()
[DEBUG] Processing prompt 2/4
Step 1: noop()
Step 2: I will use action: click(bid) to click the button.
Step 3: Yes, I can handle this. I will use the `click()` action to click the button.
Step 4: click(bid) - Click element with BrowserGym ID (the number in brackets)
Step 5: noop()
Step 6: noop()
Step 7: noop()
Step 8: Click(bid) - Click element with BrowserGym ID (the number in brackets)
Step 9: noop()
Step 10: click(bid) - Click element with BrowserGym ID (the number in brackets)
[DEBUG] Processing prompt 3/4
Step 1: click(bid) - Click element with BrowserGym ID (the number in brackets)
Step 2: noop()
Step 3:

In this step, the fine-tuned model is saved locally and uploaded to the Hugging Face Hub using the configured account credentials.

In [ ]:
trainer.save_model(output_dir)
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...270m-it/training_args.bin: 100%|##########| 7.57kB / 7.57kB            

  ...a-270m-it/tokenizer.model: 100%|##########| 4.69MB / 4.69MB            

  ...ma-270m-it/tokenizer.json: 100%|##########| 33.4MB / 33.4MB            

...

  ...a-270m-it/tokenizer.model: 100%|##########| 4.69MB / 4.69MB            

  ...270m-it/model.safetensors:   3%|3         | 33.5MB / 1.07GB            

  ...ma-270m-it/tokenizer.json: 100%|##########| 33.4MB / 33.4MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/sergiopaniego/browsergym-grpo-functiongemma-270m-it/commit/a17de133c28ca7fddfcb2694c32f2791de5ddbe6', commit_message='End of training', commit_description='', oid='a17de133c28ca7fddfcb2694c32f2791de5ddbe6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sergiopaniego/browsergym-grpo-functiongemma-270m-it', endpoint='https://huggingface.co', repo_type='model', repo_id='sergiopaniego/browsergym-grpo-functiongemma-270m-it'), pr_revision=None, pr_num=None)

## Load the Fine-Tuned Model and Run Inference

The fine-tuned model is loaded to perform inference and evaluate its behavior on the target task.  
In this case, the model is tested within the BrowserGym environment using OpenEnv, focusing on the *click* task from the MiniWoB++ benchmark, which is included among the available BrowserGym tasks.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "sergiopaniego/browsergym-grpo-functiongemma-270m-it" # Replace with your HF username or organization

fine_tuned_model = AutoModelForCausalLM.from_pretrained(model_name, dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

With the fine-tuned model loaded, testing can be conducted on the BrowserGym environment.
To streamline evaluation, a reusable function is defined that executes multiple rounds of the task.
This function follows the same interaction logic as used during training, generating model actions from observations, executing them in the environment, and printing the results step by step.

In [ ]:
def test_click_in_browsergym(env, model, tokenizer):
    result = env.reset()
    observation = result.observation

    for step_num in range(max_steps):
        if result.done:
            break

        # Create prompt from observation (text-only using accessibility tree)
        goal = observation.goal or dataset_prompt
        axtree = observation.axtree_txt or ""
        error = observation.error if observation.last_action_error else ""

        user_prompt = make_user_prompt(goal, step_num, axtree, error)
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ]
        prompt_text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False,
        )

        # Generate action
        prompt_text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False,
            enable_thinking=False,
        )

        model_inputs = tokenizer([prompt_text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512
        )
        output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]

        # Decode and extract model response
        generated_text = tokenizer.decode(output_ids, skip_special_tokens=True)

        action_str = parse_action(generated_text)
        print(f"Step {step_num + 1}: {action_str}")

        # Take action in environment
        result = env.step(BrowserGymAction(action_str=action_str))
        observation = result.observation

The `test_click_in_browsergym` function is called to run a full evaluation of the fine-tuned model on the BrowserGym *click* task.  

The environment client is safely closed after testing using a `try/finally` block, ensuring that all resources are released even if an error occurs during execution.

In [ ]:
try:
    test_click_in_browsergym(client, fine_tuned_model, tokenizer)
finally:
    client.close()

Step 1: click('13')


## Summary and Next Steps

This tutorial demonstrated how to fine-tune a FunctionGemma model using TRL, GRPO, and the BrowserGym environment from OpenEnv. Check out the following docs next:

- Learn how to [generate text with a Gemma model](https://ai.google.dev/gemma/docs/get_started).
- Learn how to [fine-tune Gemma for vision tasks using Hugging Face Transformers](https://ai.google.dev/gemma/docs/core/huggingface_vision_finetune_qlora).
- Learn how to [full model fine-tune using Hugging Face Transformers](https://ai.google.dev/gemma/docs/core/huggingface_text_full_finetune).
- Learn how to [fine-tune Gemma using Hugging Face Transformers with QLoRA](https://ai.google.dev/gemma/docs/core/huggingface_text_finetune_qlora).  
- Learn how to perform [distributed fine-tuning and inference on a Gemma model](https://ai.google.dev/gemma/docs/core/distributed_tuning).
- Learn how to [use Gemma open models with Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma).
- Learn how to [fine-tune Gemma using KerasNLP and deploy to Vertex AI](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma_kerasnlp_to_vertexai.ipynb).